In [1]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1uDFKSWW0UpRu4JZ8_ZdgsKtV7-wGMSTz'})

In [5]:
download.GetContentFile('Dataset.zip')
!unzip Dataset.zip

Archive:  Dataset.zip
   creating: Dataset/
  inflating: Dataset/zero_indexed_files.csv  
   creating: __MACOSX/
   creating: __MACOSX/Dataset/
  inflating: __MACOSX/Dataset/._zero_indexed_files.csv  
  inflating: Dataset/.DS_Store       
  inflating: __MACOSX/Dataset/._.DS_Store  
  inflating: Dataset/test_notrash.csv  
  inflating: __MACOSX/Dataset/._test_notrash.csv  
   creating: Dataset/Garbage classification/
   creating: Dataset/Garbage classification/paper/
  inflating: Dataset/Garbage classification/paper/paper283.jpg  
   creating: __MACOSX/Dataset/Garbage classification/
   creating: __MACOSX/Dataset/Garbage classification/paper/
  inflating: __MACOSX/Dataset/Garbage classification/paper/._paper283.jpg  
  inflating: Dataset/Garbage classification/paper/paper297.jpg  
  inflating: __MACOSX/Dataset/Garbage classification/paper/._paper297.jpg  
  inflating: Dataset/Garbage classification/paper/paper526.jpg  
  inflating: __MACOSX/Dataset/Garbage classification/paper/._paper526

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
%tensorflow_version 1.x
import glob, os, random

Using TensorFlow backend.


In [7]:
train = pd.read_csv('Dataset/zero_indexed_files.csv')
train.head

<bound method NDFrame.head of              image  label
0       glass1.jpg      0
1       glass2.jpg      0
2       glass3.jpg      0
3       glass4.jpg      0
4       glass5.jpg      0
...            ...    ...
2522  trash133.jpg      5
2523  trash134.jpg      5
2524  trash135.jpg      5
2525  trash136.jpg      5
2526  trash137.jpg      5

[2527 rows x 2 columns]>

In [8]:
train_image = []
for i in tqdm(range(train.shape[0])):
    if("glass" in train['image'][i]):
      img = image.load_img('Dataset/Garbage classification/glass/'+train['image'][i], target_size=(28,28,1), grayscale=False)
    elif("paper" in train['image'][i]):
      img = image.load_img('Dataset/Garbage classification/paper/'+train['image'][i], target_size=(28,28,1), grayscale=False)
    elif("cardboard" in train['image'][i]):
      img = image.load_img('Dataset/Garbage classification/cardboard/'+train['image'][i], target_size=(28,28,1), grayscale=False)
    elif("plastic" in train['image'][i]):
      img = image.load_img('Dataset/Garbage classification/plastic/'+train['image'][i], target_size=(28,28,1), grayscale=False)
    elif("metal" in train['image'][i]):
      img = image.load_img('Dataset/Garbage classification/metal/'+train['image'][i], target_size=(28,28,1), grayscale=False)
    elif("trash" in train['image'][i]):
      img = image.load_img('Dataset/Garbage classification/trash/'+train['image'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

X = np.array(train_image)

100%|██████████| 2527/2527 [00:07<00:00, 340.15it/s]


In [0]:
y=train['label'].values
y = to_categorical(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [11]:
from keras.layers import LeakyReLU
model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 2021 samples, validate on 506 samples
Epoch 1/10
2021/2021 [==============================] - 9s 5ms/step - loss: 0.4999 - acc: 0.8431 - val_loss: 1.2018 - val_acc: 0.6126
Epoch 2/10
2021/2021 [==============================] - 9s 5ms/step - loss: 0.4688 - acc: 0.8525 - val_loss: 1.4191 - val_acc: 0.5534
Epoch 3/10
2021/2021 [==============================] - 9s 5ms/step - loss: 0.4384 - acc: 0.8486 - val_loss: 1.3485 - val_acc: 0.5830
Epoch 4/10
2021/2021 [==============================] - 9s 5ms/step - loss: 0.3935 - acc: 0.8788 - val_loss: 1.5246 - val_acc: 0.5435
Epoch 5/10
2021/2021 [==============================] - 9s 5ms/step - loss: 0.3505 - acc: 0.8887 - val_loss: 1.4377 - val_acc: 0.5771
Epoch 6/10
2021/2021 [==============================] - 9s 5ms/step - loss: 0.3219 - acc: 0.8926 - val_loss: 1.4902 - val_acc: 0.5830
Epoch 7/10
2021/2021 [==============================] - 9s 5ms/step - loss: 0.2844 - acc: 0.9065 - val_loss: 1.6160 - val_acc: 0.5810
Epoch 8/10
2021

In [15]:
test_file = pd.read_csv('Dataset/test_notrash.csv')
test_file['image'][1]

'paper380.jpg'

In [16]:
test_image = []
for i in tqdm(range(test_file.shape[0])):
    if("glass" in test_file['image'][i]):
      img = image.load_img('Dataset/Garbage classification/glass/'+test_file['image'][i], target_size=(28,28,1), grayscale=False)
    elif("paper" in test_file['image'][i]):
      img = image.load_img('Dataset/Garbage classification/paper/'+test_file['image'][i], target_size=(28,28,1), grayscale=False)
    elif("cardboard" in test_file['image'][i]):
      img = image.load_img('Dataset/Garbage classification/cardboard/'+test_file['image'][i], target_size=(28,28,1), grayscale=False)
    elif("plastic" in test_file['image'][i]):
      img = image.load_img('Dataset/Garbage classification/plastic/'+test_file['image'][i], target_size=(28,28,1), grayscale=False)
    elif("metal" in test_file['image'][i]):
      img = image.load_img('Dataset/Garbage classification/metal/'+test_file['image'][i], target_size=(28,28,1), grayscale=False)
    elif("trash" in test_file['image'][i]):
      img = image.load_img('Dataset/Garbage classification/trash/'+test_file['image'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

100%|██████████| 431/431 [00:01<00:00, 345.41it/s]


In [17]:
prediction = model.predict_classes(test)
prediction

array([1, 1, 2, 0, 5, 2, 1, 1, 0, 3, 2, 0, 0, 1, 2, 3, 2, 4, 1, 2, 0, 5,
       3, 2, 3, 4, 0, 2, 2, 3, 3, 4, 4, 4, 1, 2, 3, 0, 1, 4, 2, 5, 5, 3,
       0, 0, 1, 1, 0, 1, 1, 4, 4, 4, 0, 5, 4, 4, 2, 3, 1, 1, 3, 3, 4, 1,
       1, 3, 1, 3, 4, 0, 1, 3, 3, 0, 1, 0, 4, 0, 4, 3, 2, 4, 3, 5, 5, 5,
       3, 4, 3, 3, 1, 1, 3, 5, 5, 2, 1, 3, 1, 1, 1, 2, 3, 0, 1, 0, 5, 1,
       4, 2, 1, 3, 4, 2, 2, 2, 0, 0, 1, 4, 4, 0, 3, 2, 5, 2, 1, 5, 1, 0,
       3, 0, 1, 3, 4, 0, 3, 5, 1, 4, 2, 1, 2, 3, 1, 0, 3, 2, 1, 3, 4, 0,
       0, 2, 5, 4, 4, 0, 1, 4, 0, 3, 4, 3, 1, 2, 0, 1, 2, 4, 2, 5, 3, 4,
       1, 5, 1, 5, 3, 1, 4, 0, 0, 0, 0, 1, 0, 1, 2, 4, 1, 3, 0, 3, 1, 0,
       4, 2, 0, 4, 4, 3, 0, 1, 1, 3, 1, 4, 1, 0, 5, 2, 0, 2, 0, 1, 0, 2,
       4, 3, 3, 1, 1, 5, 0, 3, 4, 3, 0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0,
       4, 3, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 1, 3, 1, 2, 3, 5, 0, 2, 4, 3,
       1, 2, 0, 2, 0, 1, 0, 3, 1, 1, 0, 1, 3, 0, 5, 1, 3, 4, 1, 1, 2, 1,
       3, 4, 0, 1, 4, 2, 4, 0, 3, 3, 3, 2, 3, 4, 3,

In [18]:
download = drive.CreateFile({'id': '1BsUTzpytyzNkuUaAA5uy-i1xR_yQy8zb'})
download.GetContentFile('test_notrash.csv')
test_file = pd.read_csv('test_notrash.csv')
test_file.head

<bound method NDFrame.head of                image  prediction
0        paper70.jpg           1
1       paper380.jpg           1
2    cardboard31.jpg           2
3        glass12.jpg           1
4       paper169.jpg           1
..               ...         ...
426     metal389.jpg           4
427     paper303.jpg           3
428     paper405.jpg           1
429     paper465.jpg           1
430     trash132.jpg           3

[431 rows x 2 columns]>

In [19]:
test_file['actual'] = ''
for i in test_file.index:
    if "glass" in test_file.loc[i,'image']:
      test_file.loc[i,'actual'] = "0"
    elif "paper" in test_file.loc[i,'image']:
      test_file.loc[i,'actual'] = "1"
    elif "cardboard" in test_file.loc[i,'image']:
      test_file.loc[i,'actual'] = "2"
    elif "plastic" in test_file.loc[i,'image']:
      test_file.loc[i,'actual'] = "3"
    elif "metal" in test_file.loc[i,'image']:
      test_file.loc[i,'actual'] = "4"
    elif "trash" in test_file.loc[i,'image']:
      test_file.loc[i,'actual'] = "5"

count = 0
for i in test_file.index:
  if(str(test_file.loc[i,'prediction']) == test_file.loc[i,'actual']):
    count = count + 1
print(count/431)

0.7262180974477959


In [20]:
test_eval = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 1.6916841711451414
Test accuracy: 0.5968379446640316
